# grixis rogue hacking

stuff for working with [this deck](https://tappedout.net/mtg-decks/19-09-19-grixis-rogues/?cb=1569124913)

In [1]:
import sys, os

sys.path.insert(0, os.path.realpath('../'))

In [2]:
import logging
import re

from mtg.cards import cards_df
from mtg.extract.tappedout import TappedoutDeck

In [3]:
deck_id = '19-09-19-grixis-rogues'
d = TappedoutDeck(deck_id=deck_id, ignore_lands=False, with_tags=True)

In [4]:
d.df.head()

,board,qty,name,printing,foil,alter,signed,condition,languange,tag_list
0,maybe,1,AEtherize,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,maybe,1,Abyssal Specter,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,maybe,1,Acquire,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,maybe,1,Adaptive Automaton,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,main,1,Akki Underminer,NaN,NaN,NaN,NaN,NaN,NaN,"[#whenever_damages_player, #whenever_damages_p..."


In [5]:
keep_cols = ['name',
             'convertedManaCost',
             'edhrecRank',
             'rarity',
             'text',
             'type', 'types', 'subtypes', 'supertypes',
             'power', 'toughness', ]

In [6]:
cards = (cards_df()
         [keep_cols]
         .groupby('name')
         .first()
         .reset_index()
         .replace({'name': {'Aetherize': 'AEtherize'}}))

## building the dataframe

In [7]:
df = (d.df
      .merge(cards,
             how='left',
             on='name')
      [['board', 'qty', 'name',] + keep_cols[1:]]
      .copy())

### adding in the rogue-specific shit I care about

#### are they a rogue?

In [8]:
has_type = df.types.notna()
df.loc[:, 'is_rogue'] = False
df.loc[has_type, 'is_rogue'] = (df
                                .loc[has_type, 'subtypes']
                                .apply(lambda t: 'Rogue' in t))

#### are they unblockable

In [9]:
def rec_is_unblockable(rec):
    # good old fashioned fuck this
    if rec['name'] == 'Glaring Spotlight':
        return False
    try:
        pattern = "(?:{}|This creature)(?:[^\.]+ and)?(?: can't be blocked| gains fear)".format(rec['name'])
        return re.search(pattern, rec.text) is not None
    except TypeError:
        return False

In [10]:
df.loc[:, 'is_unblockable'] = df.apply(func=rec_is_unblockable,
                                       axis=1)

In [11]:
is_unblockable_test_cards = ['Blighted Agent',  # regular
                             'Blind Zealot',   # infect
                             'Changeling Outcast',  # NAME ... and can't be blocked
                             'Gravelgill Duo',  # NAME ... gains fear
                             'Neurok Invisimancer', ]  # can't be blocked

assert df[df.name.isin(is_unblockable_test_cards)].is_unblockable.all()

In [12]:
not_is_unblockable_test_cards = ['Glaring Spotlight', ]

assert not df[df.name.isin(not_is_unblockable_test_cards)].is_unblockable.any()

In [13]:
# df[df.is_unblockable][['name', 'text']].head(30)

#### do they make other things unblockable

In [14]:
df.loc[:, 'makes_others_unblockable'] = ((df
                                          .text
                                          .str
                                          .contains("(?:can't be blocked|are unblockable)", regex=True)
                                          .fillna(False))
                                         & ~df.is_unblockable)

# sigh
df.loc[df.name == 'Neurok Invisimancer', 'makes_others_unblockable'] = True

In [15]:
makes_others_unblockable_test_cards = ['Aqueous Form',
                                       'Deepfathom Skulker', 
                                       'Cover of Darkness', 
                                       'Glaring Spotlight',  # creatures you control... are unblockable this turn
                                       "Joven's Tools",  # Target creature can't be blocked this turn except by Walls
                                       'Passwall Adept',
                                       'Key to the City',  # Up to one target creature can't be blocked this turn
                                       'Ethereal Usher',  # target creature is unblockable this turn
                                       'Minamo Sightbender',  # Target creature with power X or less can't be blocked this turn
                                       'Neurok Invisimancer', # When Neurok Invisimancer enters the battlefield, target creature can't be blocked this turn
                                       'Teleportal',  # Target creature you control gets +1/+0 until end of turn and can't be blocked this turn
                                       'Tetsuko Umezawa, Fugitive',
                                       "Tezzeret's Gatebreaker", 
                                       'Veiling Oddity',
                                       'Zhuge Jin, Wu Strategist', 
                                       'Deepchannel Mentor', ]  # Blue creatures you control are unblockable

test_df = df[df.name.isin(makes_others_unblockable_test_cards)]
assert test_df.makes_others_unblockable.all(), test_df[['name', 'makes_others_unblockable']]

In [16]:
df[df.name.isin(['Glaring Spotlight', 'Neurok Invisimancer'])]

,board,qty,name,convertedManaCost,edhrecRank,rarity,text,type,types,subtypes,supertypes,power,toughness,is_rogue,is_unblockable,makes_others_unblockable
140,maybe,1,Glaring Spotlight,1.0,2746.0,rare,Creatures your opponents control with hexproof...,Artifact,[Artifact],[],[],NaN,NaN,False,False,True
248,main,1,Neurok Invisimancer,3.0,7198.0,common,Neurok Invisimancer can't be blocked.\nWhen Ne...,Creature — Human Wizard,[Creature],"[Human, Wizard]",[],2,1,False,True,True


In [17]:
# df[df.is_unblockable][['name', 'text']].head(30)

#### trigger when they attack

In [18]:
def rec_has_attack_trigger(rec):
    try:
        pattern = "[Ww]henever (?:{}|[Tt]his creature)(?:[\s\w]+ or)? attacks".format(rec['name'])
        return re.search(pattern, rec.text) is not None
    except TypeError:
        return False

In [19]:
df.loc[:, 'has_attack_trigger'] = df.apply(func=rec_has_attack_trigger,
                                           axis=1)

In [20]:
has_attack_trigger_test_cards = ['Thraximundar',
                                 'Grave Titan']

assert df[df.name.isin(has_attack_trigger_test_cards)].has_attack_trigger.all()

In [21]:
# df[df.has_attack_trigger][['name', 'text']].head(30)

#### trigger when they deal damage to a creature

In [22]:
def rec_has_creature_damage_trigger(rec):
    # ugh
    if rec['name'] == 'Mephidross Vampire':
        return False
    try:
        pattern = "[Ww]henever (?:{}|[Tt]his creature|Phage) deals(?: combat)? damage to a creature".format(rec['name'])
        return re.search(pattern, rec.text) is not None
    except TypeError:
        return False

In [23]:
df.loc[:, 'has_creature_damage_trigger'] = df.apply(func=rec_has_creature_damage_trigger,
                                                    axis=1)

In [24]:
has_creature_damage_trigger_test_cards = ['Phage the Untouchable',  # stupid fucking phage
                                          'Mirri the Cursed', ]

assert df[df.name.isin(has_creature_damage_trigger_test_cards)].has_creature_damage_trigger.all()

assert not df[df.name == 'Mephidross Vampire'].has_creature_damage_trigger.iloc[0]

In [25]:
# df[df.has_creature_damage_trigger][['name', 'text']].head(30)

#### trigger when they deal damage to a player

In [26]:
def rec_has_player_damage_trigger(rec):
    try:
        pattern = "[Ww]henever (?:{}|[Tt]his creature|)(?:\w+)? deals(?: combat)? damage to (?:a player|an opponent)"
        pattern = pattern.format(rec['name'])
        return re.search(pattern, rec.text) is not None
    except TypeError:
        return False

In [27]:
df.loc[:, 'has_player_damage_trigger'] = df.apply(func=rec_has_player_damage_trigger,
                                                  axis=1)

In [28]:
has_player_damage_trigger_test_cards = ['Garza Vol, Plague Queen', 
                                        'Abyssal Specter',
                                        'Hypnotic Specter']

assert df[df.name.isin(has_player_damage_trigger_test_cards)].has_player_damage_trigger.all()

In [29]:
# df[df.has_player_damage_trigger][['name', 'text']].head(30)

#### trigger when others attack

In [30]:
df.loc[:, 'has_others_attack_trigger'] = ((df.text.str.contains('Whenever')
                                           & df.text.str.contains('attacks')
                                           & ~df.has_attack_trigger)
                                          .fillna(False))

In [31]:
has_others_trigger_test_cards = ['Acqueous Form',
                                 'Sword of the Animist']

assert df[df.name.isin(has_others_trigger_test_cards)].has_others_attack_trigger.all()

In [32]:
df[df.has_others_attack_trigger][['name', 'text']].head(30)

,name,text
6,Aqueous Form,Enchant creature\nEnchanted creature can't be ...
38,Cavalcade of Calamity,Whenever a creature you control with power 1 o...
122,Explorer's Scope,"Whenever equipped creature attacks, look at th..."
164,Hellrider,Haste\nWhenever a creature you control attacks...
189,Kindred Discovery,"As Kindred Discovery enters the battlefield, c..."
210,Marchesa's Decree,"When Marchesa's Decree enters the battlefield,..."
234,Moonsilver Spear,Equipped creature has first strike.\nWhenever ...
294,Raid Bombardment,Whenever a creature you control with power 2 o...
301,Raving Dead,Deathtouch\nAt the beginning of combat on your...
345,Stinkdrinker Bandit,Prowl {1}{B} (You may cast this for its prowl ...


#### trigger when others deal damage to a creature

In [33]:
df.loc[:, 'has_others_creature_damage_trigger'] = (((df.text.str.contains('Whenever')
                                                     & (df.text.str.contains('deals combat damage to a creature')
                                                        | df.text.str.contains('deals combat damage to an opponent'))
                                                     & ~df.has_creature_damage_trigger)
                                                    | df.name.isin(['Mephidross Vampire', 'Neko-Te']))
                                                   .fillna(False))

In [34]:
has_others_creature_trigger_test_cards = ['Mephidross Vampire',
                                          'Neko-Te', ]

assert df[df.name.isin(has_others_creature_trigger_test_cards)].has_others_creature_damage_trigger.all()

In [35]:
# df[df.has_others_creature_damage_trigger][['name', 'text']].head(30)

#### trigger when others deal damage to a person

In [36]:
df.loc[:, 'has_others_player_damage_trigger'] = ((df.text.str.contains('Whenever')
                                                  & df.text.str.contains('deals combat damage to a player')
                                                  & ~df.has_player_damage_trigger)
                                                 .fillna(False))

In [37]:
has_others_players_trigger_test_cards = ["Oona's Blackguard", 
                                         "Auntie's Snitch",
                                         "Bident of Thassa"
                                         "Sword of Light and Shadow", ]

assert df[df.name.isin(has_others_players_trigger_test_cards)].has_others_player_damage_trigger.all()

In [38]:
df[df.has_others_player_damage_trigger][['name', 'text']].head(30)

,name,text
11,Auntie's Snitch,Auntie's Snitch can't block.\nProwl {1}{B} (Yo...
16,Bident of Thassa,Whenever a creature you control deals combat d...
26,Bloodforged Battle-Axe,Equipped creature gets +2/+0.\nWhenever equipp...
36,Call of the Nightwing,Create a 1/1 blue and black Horror creature to...
79,Deepfathom Skulker,Devoid (This card has no color.)\nWhenever a c...
82,Destructive Urge,Enchant creature\nWhenever enchanted creature ...
106,Dowsing Dagger,"When Dowsing Dagger enters the battlefield, ta..."
135,"Garza Zol, Plague Queen","Flying, haste\nWhenever a creature dealt damag..."
154,"Grenzo, Havoc Raiser",Whenever a creature you control deals combat d...
158,Hands of Binding,Tap target creature an opponent controls. That...


In [39]:
df.head()

,board,qty,name,convertedManaCost,edhrecRank,rarity,text,type,types,subtypes,...,toughness,is_rogue,is_unblockable,makes_others_unblockable,has_attack_trigger,has_creature_damage_trigger,has_player_damage_trigger,has_others_attack_trigger,has_others_creature_damage_trigger,has_others_player_damage_trigger
0,maybe,1,AEtherize,4.0,542.0,uncommon,Return all attacking creatures to their owner'...,Instant,[Instant],[],...,NaN,False,False,False,False,False,False,False,False,False
1,maybe,1,Abyssal Specter,4.0,7210.0,uncommon,Flying\nWhenever Abyssal Specter deals damage ...,Creature — Specter,[Creature],[Specter],...,3,False,False,False,False,False,True,False,False,False
2,maybe,1,Acquire,5.0,2868.0,rare,Search target opponent's library for an artifa...,Sorcery,[Sorcery],[],...,NaN,False,False,False,False,False,False,False,False,False
3,maybe,1,Adaptive Automaton,3.0,968.0,rare,"As Adaptive Automaton enters the battlefield, ...",Artifact Creature — Construct,"[Artifact, Creature]",[Construct],...,2,False,False,False,False,False,False,False,False,False
4,main,1,Akki Underminer,4.0,11418.0,uncommon,Whenever Akki Underminer deals combat damage t...,Creature — Goblin Rogue Shaman,[Creature],"[Goblin, Rogue, Shaman]",...,1,True,False,False,False,False,True,False,False,False


In [40]:
df.to_csv('grixis-rogues.csv')